In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 10
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
    
class ModelConfig(object):
    embeddingSize = 200
    
    filters = 128  # 内层一维卷积核的数量，外层卷积核的数量应该等于embeddingSize，因为要确保每个layer后的输出维度和输入维度是一致的。
    numHeads = 8  # Attention 的头数
    numBlocks = 1  # 设置transformer block的数量
    epsilon = 1e-8  # LayerNorm 层中的最小除数
    keepProp = 0.9  # multi head attention 中的dropout
    
    dropoutKeepProb = 0.5 # 全连接层的dropout
    l2RegLambda = 0.0
    
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
# 生成位置嵌入
def fixedPositionEmbedding(batchSize, sequenceLen):
    embeddedPosition = []
    for batch in range(batchSize):
        x = []
        for step in range(sequenceLen):
            a = np.zeros(sequenceLen)
            a[step] = 1
            x.append(a)
        embeddedPosition.append(x)
    
    return np.array(embeddedPosition, dtype="float32")

In [7]:
# 模型构建

class Transformer(object):
    """
    Transformer Encoder 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        self.embeddedPosition = tf.placeholder(tf.float32, [None, config.sequenceLength, config.sequenceLength], name="embeddedPosition")
        
        self.config = config
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层, 位置向量的定义方式有两种：一是直接用固定的one-hot的形式传入，然后和词向量拼接，在当前的数据集上表现效果更好。另一种
        # 就是按照论文中的方法实现，这样的效果反而更差，可能是增大了模型的复杂度，在小数据集上表现不佳。
        
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embedded = tf.nn.embedding_lookup(self.W, self.inputX)
            self.embeddedWords = tf.concat([self.embedded, self.embeddedPosition], -1)

        with tf.name_scope("transformer"):
            for i in range(config.model.numBlocks):
                with tf.name_scope("transformer-{}".format(i + 1)):
            
                    # 维度[batch_size, sequence_length, embedding_size]
                    multiHeadAtt = self._multiheadAttention(rawKeys=self.embedded, queries=self.embeddedWords,
                                                            keys=self.embeddedWords)
                    # 维度[batch_size, sequence_length, embedding_size]
                    self.embeddedWords = self._feedForward(multiHeadAtt, 
                                                           [config.model.filters, config.model.embeddingSize + config.sequenceLength])
                
            outputs = tf.reshape(self.embeddedWords, [-1, config.sequenceLength * (config.model.embeddingSize + config.sequenceLength)])

        outputSize = outputs.get_shape()[-1].value

#         with tf.name_scope("wordEmbedding"):
#             self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec"), name="W")
#             self.wordEmbedded = tf.nn.embedding_lookup(self.W, self.inputX)
        
#         with tf.name_scope("positionEmbedding"):
#             print(self.wordEmbedded)
#             self.positionEmbedded = self._positionEmbedding()
            
#         self.embeddedWords = self.wordEmbedded + self.positionEmbedded
            
#         with tf.name_scope("transformer"):
#             for i in range(config.model.numBlocks):
#                 with tf.name_scope("transformer-{}".format(i + 1)):
            
#                     # 维度[batch_size, sequence_length, embedding_size]
#                     multiHeadAtt = self._multiheadAttention(rawKeys=self.wordEmbedded, queries=self.embeddedWords,
#                                                             keys=self.embeddedWords)
#                     # 维度[batch_size, sequence_length, embedding_size]
#                     self.embeddedWords = self._feedForward(multiHeadAtt, [config.model.filters, config.model.embeddingSize])
                
#             outputs = tf.reshape(self.embeddedWords, [-1, config.sequenceLength * (config.model.embeddingSize)])

#         outputSize = outputs.get_shape()[-1].value
        
        with tf.name_scope("dropout"):
            outputs = tf.nn.dropout(outputs, keep_prob=self.dropoutKeepProb)
    
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(outputs, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss
            
    def _layerNormalization(self, inputs, scope="layerNorm"):
        # LayerNorm层和BN层有所不同
        epsilon = self.config.model.epsilon

        inputsShape = inputs.get_shape() # [batch_size, sequence_length, embedding_size]

        paramsShape = inputsShape[-1:]

        # LayerNorm是在最后的维度上计算输入的数据的均值和方差，BN层是考虑所有维度的
        # mean, variance的维度都是[batch_size, sequence_len, 1]
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)

        beta = tf.Variable(tf.zeros(paramsShape))

        gamma = tf.Variable(tf.ones(paramsShape))
        normalized = (inputs - mean) / ((variance + epsilon) ** .5)
        
        outputs = gamma * normalized + beta

        return outputs
            
    def _multiheadAttention(self, rawKeys, queries, keys, numUnits=None, causality=False, scope="multiheadAttention"):
        # rawKeys 的作用是为了计算mask时用的，因为keys是加上了position embedding的，其中不存在padding为0的值
        
        numHeads = self.config.model.numHeads
        keepProp = self.config.model.keepProp
        
        if numUnits is None:  # 若是没传入值，直接去输入数据的最后一维，即embedding size.
            numUnits = queries.get_shape().as_list()[-1]

        # tf.layers.dense可以做多维tensor数据的非线性映射，在计算self-Attention时，一定要对这三个值进行非线性映射，
        # 其实这一步就是论文中Multi-Head Attention中的对分割后的数据进行权重映射的步骤，我们在这里先映射后分割，原则上是一样的。
        # Q, K, V的维度都是[batch_size, sequence_length, embedding_size]
        Q = tf.layers.dense(queries, numUnits, activation=tf.nn.relu)
        K = tf.layers.dense(keys, numUnits, activation=tf.nn.relu)
        V = tf.layers.dense(keys, numUnits, activation=tf.nn.relu)

        # 将数据按最后一维分割成num_heads个, 然后按照第一维拼接
        # Q, K, V 的维度都是[batch_size * numHeads, sequence_length, embedding_size/numHeads]
        Q_ = tf.concat(tf.split(Q, numHeads, axis=-1), axis=0) 
        K_ = tf.concat(tf.split(K, numHeads, axis=-1), axis=0) 
        V_ = tf.concat(tf.split(V, numHeads, axis=-1), axis=0)

        # 计算keys和queries之间的点积，维度[batch_size * numHeads, queries_len, key_len], 后两维是queries和keys的序列长度
        similary = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1]))

        # 对计算的点积进行缩放处理，除以向量长度的根号值
        scaledSimilary = similary / (K_.get_shape().as_list()[-1] ** 0.5)

        # 在我们输入的序列中会存在padding这个样的填充词，这种词应该对最终的结果是毫无帮助的，原则上说当padding都是输入0时，
        # 计算出来的权重应该也是0，但是在transformer中引入了位置向量，当和位置向量相加之后，其值就不为0了，因此在添加位置向量
        # 之前，我们需要将其mask为0。虽然在queries中也存在这样的填充词，但原则上模型的结果之和输入有关，而且在self-Attention中
        # queryies = keys，因此只要一方为0，计算出的权重就为0。
        # 具体关于key mask的介绍可以看看这里： https://github.com/Kyubyong/transformer/issues/3

        # 将每一时序上的向量中的值相加取平均值
        keyMasks = tf.sign(tf.abs(tf.reduce_sum(rawKeys, axis=-1)))  # 维度[batch_size, time_step]

        # 利用tf，tile进行张量扩张， 维度[batch_size * numHeads, keys_len] keys_len = keys 的序列长度
        keyMasks = tf.tile(keyMasks, [numHeads, 1]) 

        # 增加一个维度，并进行扩张，得到维度[batch_size * numHeads, queries_len, keys_len]
        keyMasks = tf.tile(tf.expand_dims(keyMasks, 1), [1, tf.shape(queries)[1], 1])

        # tf.ones_like生成元素全为1，维度和scaledSimilary相同的tensor, 然后得到负无穷大的值
        paddings = tf.ones_like(scaledSimilary) * (-2 ** (32 + 1))

        # tf.where(condition, x, y),condition中的元素为bool值，其中对应的True用x中的元素替换，对应的False用y中的元素替换
        # 因此condition,x,y的维度是一样的。下面就是keyMasks中的值为0就用paddings中的值替换
        maskedSimilary = tf.where(tf.equal(keyMasks, 0), paddings, scaledSimilary) # 维度[batch_size * numHeads, queries_len, key_len]

        # 在计算当前的词时，只考虑上文，不考虑下文，出现在Transformer Decoder中。在文本分类时，可以只用Transformer Encoder。
        # Decoder是生成模型，主要用在语言生成中
        if causality:
            diagVals = tf.ones_like(maskedSimilary[0, :, :])  # [queries_len, keys_len]
            tril = tf.contrib.linalg.LinearOperatorTriL(diagVals).to_dense()  # [queries_len, keys_len]
            masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(maskedSimilary)[0], 1, 1])  # [batch_size * numHeads, queries_len, keys_len]

            paddings = tf.ones_like(masks) * (-2 ** (32 + 1))
            maskedSimilary = tf.where(tf.equal(masks, 0), paddings, maskedSimilary)  # [batch_size * numHeads, queries_len, keys_len]

        # 通过softmax计算权重系数，维度 [batch_size * numHeads, queries_len, keys_len]
        weights = tf.nn.softmax(maskedSimilary)

        # 加权和得到输出值, 维度[batch_size * numHeads, sequence_length, embedding_size/numHeads]
        outputs = tf.matmul(weights, V_)

        # 将多头Attention计算的得到的输出重组成最初的维度[batch_size, sequence_length, embedding_size]
        outputs = tf.concat(tf.split(outputs, numHeads, axis=0), axis=2)
        
        outputs = tf.nn.dropout(outputs, keep_prob=keepProp)

        # 对每个subLayers建立残差连接，即H(x) = F(x) + x
        outputs += queries
        # normalization 层
        outputs = self._layerNormalization(outputs)
        return outputs

    def _feedForward(self, inputs, filters, scope="multiheadAttention"):
        # 在这里的前向传播采用卷积神经网络
        
        # 内层
        params = {"inputs": inputs, "filters": filters[0], "kernel_size": 1,
                  "activation": tf.nn.relu, "use_bias": True}
        outputs = tf.layers.conv1d(**params)

        # 外层
        params = {"inputs": outputs, "filters": filters[1], "kernel_size": 1,
                  "activation": None, "use_bias": True}

        # 这里用到了一维卷积，实际上卷积核尺寸还是二维的，只是只需要指定高度，宽度和embedding size的尺寸一致
        # 维度[batch_size, sequence_length, embedding_size]
        outputs = tf.layers.conv1d(**params)

        # 残差连接
        outputs += inputs

        # 归一化处理
        outputs = self._layerNormalization(outputs)

        return outputs
    
    def _positionEmbedding(self, scope="positionEmbedding"):
        # 生成可训练的位置向量
        batchSize = self.config.batchSize
        sequenceLen = self.config.sequenceLength
        embeddingSize = self.config.model.embeddingSize
        
        # 生成位置的索引，并扩张到batch中所有的样本上
        positionIndex = tf.tile(tf.expand_dims(tf.range(sequenceLen), 0), [batchSize, 1])

        # 根据正弦和余弦函数来获得每个位置上的embedding的第一部分
        positionEmbedding = np.array([[pos / np.power(10000, (i-i%2) / embeddingSize) for i in range(embeddingSize)] 
                                      for pos in range(sequenceLen)])

        # 然后根据奇偶性分别用sin和cos函数来包装
        positionEmbedding[:, 0::2] = np.sin(positionEmbedding[:, 0::2])
        positionEmbedding[:, 1::2] = np.cos(positionEmbedding[:, 1::2])

        # 将positionEmbedding转换成tensor的格式
        positionEmbedding_ = tf.cast(positionEmbedding, dtype=tf.float32)

        # 得到三维的矩阵[batchSize, sequenceLen, embeddingSize]
        positionEmbedded = tf.nn.embedding_lookup(positionEmbedding_, positionIndex)

        return positionEmbedded
            

In [8]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [9]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

embeddedPosition = fixedPositionEmbedding(config.batchSize, config.sequenceLength)

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        transformer = Transformer(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(transformer.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", transformer.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/Transformer/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              transformer.inputX: batchX,
              transformer.inputY: batchY,
              transformer.dropoutKeepProb: config.model.dropoutKeepProb,
              transformer.embeddedPosition: embeddedPosition
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, transformer.loss, transformer.predictions, transformer.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              transformer.inputX: batchX,
              transformer.inputY: batchY,
              transformer.dropoutKeepProb: 1.0,
              transformer.embeddedPosition: embeddedPosition
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, transformer.loss, transformer.predictions, transformer.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/Transformer/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(transformer.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(transformer.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(transformer.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name dense/kernel:0/grad/hist is illegal; using dense/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name dense/kernel:0/grad/sparsity is illegal; using dense/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name dense/bias:0/grad/hist is illegal; using dense/bias_0/grad/hist instead.
INFO:tensorflow:Summary name dense/bias:0/grad/sparsity is illegal; using dense/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name dense_1/kernel:0/grad/hist is illegal; using dense_1/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name dense_1/kernel:0/grad/sparsity is illegal; using dense_1/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name dense_1/bias:0/grad/hist is illegal; using dense_1/bias_0/grad/hist instead.
INFO:tensorflow:Su

2019-01-14T17:47:14.107319, step: 34, loss: 1.0067329406738281, acc: 0.6094, auc: 0.6293, precision: 0.6234, recall: 0.6957
2019-01-14T17:47:14.330665, step: 35, loss: 1.0151830911636353, acc: 0.5625, auc: 0.5846, precision: 0.5429, recall: 0.322
2019-01-14T17:47:14.556159, step: 36, loss: 0.8511122465133667, acc: 0.6562, auc: 0.7115, precision: 0.6786, recall: 0.3519
2019-01-14T17:47:14.775648, step: 37, loss: 1.3129496574401855, acc: 0.5312, auc: 0.6156, precision: 0.6667, recall: 0.1538
2019-01-14T17:47:14.995183, step: 38, loss: 1.041608214378357, acc: 0.5781, auc: 0.5918, precision: 0.4565, recall: 0.42
2019-01-14T17:47:15.215155, step: 39, loss: 1.0777957439422607, acc: 0.5938, auc: 0.6068, precision: 0.597, recall: 0.6154
2019-01-14T17:47:15.440356, step: 40, loss: 1.0155107975006104, acc: 0.5938, auc: 0.623, precision: 0.5867, recall: 0.6769
2019-01-14T17:47:15.661437, step: 41, loss: 0.8809636235237122, acc: 0.6172, auc: 0.6727, precision: 0.6721, recall: 0.5857
2019-01-14T17:

2019-01-14T17:47:38.052782, step: 100, loss: 0.4927764603724846, acc: 0.8243256410256412, auc: 0.9054974358974357, precision: 0.8074358974358974, recall: 0.8587153846153845
2019-01-14T17:47:38.267294, step: 101, loss: 0.6378569602966309, acc: 0.8047, auc: 0.8796, precision: 0.7778, recall: 0.8615
2019-01-14T17:47:38.486141, step: 102, loss: 0.689957857131958, acc: 0.7969, auc: 0.8988, precision: 0.697, recall: 0.8846
2019-01-14T17:47:38.708786, step: 103, loss: 0.5570914149284363, acc: 0.8047, auc: 0.8889, precision: 0.8621, recall: 0.7463
2019-01-14T17:47:38.934067, step: 104, loss: 0.660875141620636, acc: 0.7812, auc: 0.8881, precision: 0.8772, recall: 0.7042
2019-01-14T17:47:39.155304, step: 105, loss: 0.5929709672927856, acc: 0.7266, auc: 0.8971, precision: 0.8491, recall: 0.625
2019-01-14T17:47:39.374979, step: 106, loss: 0.6518077254295349, acc: 0.8125, auc: 0.8696, precision: 0.7895, recall: 0.8824
2019-01-14T17:47:39.594123, step: 107, loss: 0.8777315020561218, acc: 0.7578, auc

2019-01-14T17:47:52.856468, step: 166, loss: 0.8000973463058472, acc: 0.7656, auc: 0.9108, precision: 0.6951, recall: 0.9194
2019-01-14T17:47:53.083006, step: 167, loss: 0.5640276074409485, acc: 0.8359, auc: 0.9384, precision: 0.7733, recall: 0.9355
2019-01-14T17:47:53.302194, step: 168, loss: 0.35207679867744446, acc: 0.8594, auc: 0.9422, precision: 0.8955, recall: 0.8451
2019-01-14T17:47:53.516404, step: 169, loss: 0.3525891900062561, acc: 0.8203, auc: 0.9538, precision: 0.8704, recall: 0.746
2019-01-14T17:47:53.733768, step: 170, loss: 0.42336422204971313, acc: 0.8125, auc: 0.9831, precision: 1.0, recall: 0.6066
2019-01-14T17:47:53.944170, step: 171, loss: 0.26315364241600037, acc: 0.8828, auc: 0.9655, precision: 0.9388, recall: 0.7931
2019-01-14T17:47:54.174560, step: 172, loss: 0.5387764573097229, acc: 0.8125, auc: 0.9114, precision: 0.8028, recall: 0.8507
2019-01-14T17:47:54.409961, step: 173, loss: 0.4923793077468872, acc: 0.8516, auc: 0.9399, precision: 0.8228, recall: 0.9286
2

2019-01-14T17:48:15.723874, step: 231, loss: 0.3040640652179718, acc: 0.8984, auc: 0.9582, precision: 0.8857, recall: 0.9254
2019-01-14T17:48:15.950867, step: 232, loss: 0.31216657161712646, acc: 0.8984, auc: 0.9485, precision: 0.9322, recall: 0.8594
2019-01-14T17:48:16.179284, step: 233, loss: 0.45784682035446167, acc: 0.8125, auc: 0.9295, precision: 0.8776, recall: 0.7049
2019-01-14T17:48:16.407975, step: 234, loss: 0.33681797981262207, acc: 0.8672, auc: 0.9514, precision: 0.9091, recall: 0.8065
2019-01-14T17:48:16.618809, step: 235, loss: 0.4005366861820221, acc: 0.8125, auc: 0.9291, precision: 0.82, recall: 0.7321
2019-01-14T17:48:16.837597, step: 236, loss: 0.3651196360588074, acc: 0.8594, auc: 0.9466, precision: 0.8906, recall: 0.8382
2019-01-14T17:48:17.053995, step: 237, loss: 0.4534922242164612, acc: 0.8359, auc: 0.9279, precision: 0.8254, recall: 0.8387
2019-01-14T17:48:17.282156, step: 238, loss: 0.2123047262430191, acc: 0.9062, auc: 0.9714, precision: 0.9062, recall: 0.9062

...

2019-01-14T17:53:27.583158, step: 1229, loss: 0.006735920440405607, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:53:27.805415, step: 1230, loss: 0.02247035689651966, acc: 0.9922, auc: 1.0, precision: 0.9855, recall: 1.0
2019-01-14T17:53:28.020289, step: 1231, loss: 0.006388053297996521, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9853
2019-01-14T17:53:28.234148, step: 1232, loss: 0.040617913007736206, acc: 0.9922, auc: 0.9998, precision: 0.9844, recall: 1.0
2019-01-14T17:53:28.453678, step: 1233, loss: 0.008222617208957672, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9859
2019-01-14T17:53:28.669432, step: 1234, loss: 0.013482476584613323, acc: 0.9922, auc: 0.9998, precision: 1.0, recall: 0.9821
2019-01-14T17:53:28.883389, step: 1235, loss: 0.006248192861676216, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:53:29.111260, step: 1236, loss: 0.014908703044056892, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9846
2019-01-14T17:53:29.331358, ste

2019-01-14T17:53:42.966080, step: 1298, loss: 0.008322563953697681, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9828
2019-01-14T17:53:43.179891, step: 1299, loss: 0.0007304264581762254, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:53:43.405649, step: 1300, loss: 0.02242034487426281, acc: 0.9922, auc: 0.9995, precision: 0.9839, recall: 1.0

Evaluation:
2019-01-14T17:53:51.850018, step: 1300, loss: 0.8448327519954779, acc: 0.8603846153846155, auc: 0.937220512820513, precision: 0.8759051282051282, recall: 0.8436948717948716
2019-01-14T17:53:52.062701, step: 1301, loss: 0.0004344551998656243, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:53:52.286373, step: 1302, loss: 0.0006297562504187226, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:53:52.510232, step: 1303, loss: 0.004761956166476011, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:53:52.736799, step: 1304, loss: 0.018704865127801895, acc: 0.9922, auc: 1.0, precision:

2019-01-14T17:54:06.493783, step: 1366, loss: 0.005980116315186024, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:06.713703, step: 1367, loss: 0.018088338896632195, acc: 0.9922, auc: 1.0, precision: 0.9859, recall: 1.0
2019-01-14T17:54:06.932027, step: 1368, loss: 0.031985241919755936, acc: 0.9844, auc: 0.9998, precision: 0.9833, recall: 0.9833
2019-01-14T17:54:07.163188, step: 1369, loss: 0.011684699915349483, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9853
2019-01-14T17:54:07.370305, step: 1370, loss: 0.021527869626879692, acc: 0.9844, auc: 1.0, precision: 1.0, recall: 0.9672
2019-01-14T17:54:07.584448, step: 1371, loss: 0.0016484045190736651, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:07.792972, step: 1372, loss: 0.05587197095155716, acc: 0.9766, auc: 1.0, precision: 1.0, recall: 0.9545
2019-01-14T17:54:08.031872, step: 1373, loss: 0.00252883811481297, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:08.258198, step: 137

2019-01-14T17:54:29.997049, step: 1434, loss: 0.00553515600040555, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9821
2019-01-14T17:54:30.225951, step: 1435, loss: 0.000726417638361454, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:30.461458, step: 1436, loss: 0.004697820171713829, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:30.700395, step: 1437, loss: 0.00018625099619384855, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:30.937163, step: 1438, loss: 0.0028216582722961903, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:31.171882, step: 1439, loss: 0.013424006290733814, acc: 0.9922, auc: 1.0, precision: 0.9857, recall: 1.0
2019-01-14T17:54:31.410873, step: 1440, loss: 0.0007932220469228923, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:31.638381, step: 1441, loss: 0.006314391270279884, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9821
2019-01-14T17:54:31.865662, step: 1442, loss: 0.006

2019-01-14T17:54:53.849653, step: 1502, loss: 0.01015173178166151, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9839
2019-01-14T17:54:54.074385, step: 1503, loss: 0.012614555656909943, acc: 0.9844, auc: 1.0, precision: 1.0, recall: 0.9701
2019-01-14T17:54:54.296632, step: 1504, loss: 0.0005226375651545823, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:54.523309, step: 1505, loss: 0.0016432186821475625, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:54.753028, step: 1506, loss: 0.023906569927930832, acc: 0.9922, auc: 1.0, precision: 0.9831, recall: 1.0
2019-01-14T17:54:54.975920, step: 1507, loss: 0.006000826135277748, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:55.205114, step: 1508, loss: 0.0051110973581671715, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-14T17:54:55.426481, step: 1509, loss: 0.02013910748064518, acc: 0.9844, auc: 0.9998, precision: 0.9846, recall: 0.9846
2019-01-14T17:54:55.652688, step: 1510, l